In [15]:
import numpy as np
from pymoo.factory import get_problem, get_visualization
from scipy.stats import qmc
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
import pandas as pd

seed = 1

problem = get_problem("ackley", n_var=2, a=20, b=1/5, c=2 * np.pi)
optimizer = GA(pop_size = 10)
res = minimize(problem, optimizer, termination=("n_eval", 100), seed=seed, verbose = True)
np.set_printoptions(formatter={"all": lambda x: f"{x:f}"})
print(res.X, res.F)



lhs = qmc.LatinHypercube(problem.n_var, seed=seed)
X = lhs.random(50)
X = qmc.scale(X, problem.xl, problem.xu)
y = problem.evaluate(X)

df = pd.DataFrame(X)
# print(df.describe())



n_gen |  n_eval |     fopt     |   fopt_gap   |     favg    
    1 |      10 |  1.40464E+01 |  1.40464E+01 |  1.94662E+01
    2 |      20 |  1.27105E+01 |  1.27105E+01 |  1.69706E+01
    3 |      30 |  1.27105E+01 |  1.27105E+01 |  1.56481E+01
    4 |      40 |  1.27105E+01 |  1.27105E+01 |  1.40725E+01
    5 |      50 |  1.27105E+01 |  1.27105E+01 |  1.34610E+01
    6 |      60 |  1.26023E+01 |  1.26023E+01 |  1.31202E+01
    7 |      70 |  1.25642E+01 |  1.25642E+01 |  1.29883E+01
    8 |      80 |  1.08305E+01 |  1.08305E+01 |  1.26062E+01
    9 |      90 |  1.08305E+01 |  1.08305E+01 |  1.24891E+01
   10 |     100 |  1.01938E+01 |  1.01938E+01 |  1.21731E+01
[4.236416 0.973671] [10.193839]
